This setup expects samples to be in the ``data_training`` directory in the following structure (``*_mmap`` directories with ``RaggedMmap`` objects storing computed features)

```
data_training >
    testing >
        unknown >
            negative_batch_1_mmap
        wakeword >
            positive_batch_1_mmap
    training >
        unknown >
            negative_train_1_mmap
        wakeword >
            positive_train_1_mmap
    validation >
        unknown >
            negative_validation_1_mmap
        wakeword >
            positive_validation_1_mmap
```

In [ ]:
TRAIN_DIR = 'trained_models/inception'

SAMPLE_RATE = 16000
CLIP_DURATION_MS = 1490
FEATURE_BIN_COUNT = 40
TIME_SHIFT_MS = 0.0
WINDOW_STRIDE = 20
WINDOW_SIZE_MS = 30
PREPROCESS = 'none'
WANTED_WORDS = "wakeword,unknown"
DATASET_DIR =  'data_training'

In [ ]:
!python -m kws_streaming.train.model_train_eval \
--data_url='' \
--data_dir={DATASET_DIR} \
--train_dir={TRAIN_DIR} \
--split_data 0 \
--mel_upper_edge_hertz 7500.0 \
--mel_lower_edge_hertz 125.0 \
--how_many_training_steps 10000 \
--learning_rate 0.001 \
--window_size_ms={WINDOW_SIZE_MS} \
--window_stride_ms={WINDOW_STRIDE} \
--clip_duration_ms={CLIP_DURATION_MS} \
--eval_step_interval=500 \
--mel_num_bins={FEATURE_BIN_COUNT} \
--dct_num_features={FEATURE_BIN_COUNT} \
--preprocess={PREPROCESS} \
--alsologtostderr \
--train 1 \
--wanted_words={WANTED_WORDS} \
--pick_deterministically 0 \
--return_softmax 1 \
--restore_checkpoint 0 \
inception \
--cnn1_filters '32' \
--cnn1_kernel_sizes '5' \
--cnn1_strides '1' \
--cnn2_filters1 '16,16,16' \
--cnn2_filters2 '32,64,70' \
--cnn2_kernel_sizes '3,5,5' \
--cnn2_strides '1,1,1' \
--dropout 0.0 \
--bn_scale 0

In [ ]:
# converts the saved internal streaming model to a quantified tflite model
# requires a representative dataset for the generated features in the file representative_dataset.txt
# Must be run after restarting the interpreter, as it needs to clear the old sessions and run in eager mode

import tensorflow as tf
import numpy as np
import os


features = np.load('hey_jarvis_live_features.npy') # features generated from a short sample with the wake word said once

features[0,0] = 26.0  # guarantee one pixel is the preprocessor max

def representative_dataset_gen():
  for i in range(features.shape[0]):
    yield [features[i,:]]

model_path = os.path.join('trained_models', 'inception','stream_state_internal')
converter = tf.compat.v2.lite.TFLiteConverter.from_saved_model(model_path)

converter.experimental_new_quantizer = True
converter.experimental_enable_resource_variables = True
converter.experimental_new_converter = True
converter._experimental_variable_quantization = True

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.inference_type=tf.int8
converter.inference_input_type=tf.int8
converter.inference_output_type=tf.uint8

converter.representative_dataset = representative_dataset_gen

tflite_model = converter.convert()

saved_tflite_path = os.path.join('trained_models', 'inception', 'stream_state_internal_quantized.tflite')
tflite_model_size = open(saved_tflite_path, "wb").write(tflite_model)
print("Quantized model is %d bytes" % tflite_model_size)

In [ ]:
saved_tflite_path = os.path.join('trained_models', 'inception', 'stream_state_internal_quantized.tflite')
tflite_flat_buffer_path = os.path.join('trained_models', 'inception', 'stream_state_internal_quantized.cc')

!xxd -i {saved_tflite_path} \
> {tflite_flat_buffer_path}